In [1]:
import os, time, shutil

def crawl_directory(dirname):
    """ Walk a nested directory to get all filename ending in a pattern """
    for path, subdirs, files in os.walk(dirname):
        for name in files:
            if not name.endswith('.DS_Store'):
                yield os.path.join(path, name)


def remove_empty_dirs(path):
    for root, dirnames, filenames in os.walk(path, topdown=False):
        for dirname in dirnames:
            remove_empty_dir(os.path.realpath(os.path.join(root, dirname)))


def remove_empty_dir(path):
    try:
        os.rmdir(path)
    except OSError:
        pass

In [2]:
def merge_mixture(dirname):
    outdir = dirname
    for idx, file in enumerate(os.listdir(dirname)):

        if '.DS_Store' in file:
            continue

        for nest in crawl_directory(dirname + file):

            index = 1

            if 'dims' not in nest.split('/')[7]:
                outdir = '/'.join(nest.split('/')[:7] + nest.split('/')[8:9]) + '/'
            else:
                outdir = dirname
                
            # Initialize directory
            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            try:
                shutil.move(nest, outdir + 'trial_{0}/'.format(index))
            except:
                extension = nest.split('/')[-1]
                while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                    index += 1

                if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                    os.makedirs(outdir + 'trial_{0}/'.format(index))

                shutil.move(nest, outdir + 'trial_{0}/'.format(index))

    remove_empty_dirs(dirname)
    
    
def merge_multivariate(dirname):
    outdir = dirname
    for idx, file in enumerate(os.listdir(dirname)):

        if '.DS_Store' in file:
            continue

        for nest in crawl_directory(dirname + file):

            index = 1

            if 'dims' not in nest.split('/')[6]:
                outdir = '/'.join(nest.split('/')[:7] + nest.split('/')[8:9]) + '/'
            else:
                # Uncomment the + for mixture
                outdir = dirname + nest.split('/')[6] + '/'
                
            # Initialize directory
            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            try:
                shutil.move(nest, outdir + 'trial_{0}/'.format(index))
            except:
                extension = nest.split('/')[-1]
                while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                    index += 1

                if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                    os.makedirs(outdir + 'trial_{0}/'.format(index))

                shutil.move(nest, outdir + 'trial_{0}/'.format(index))

    remove_empty_dirs(dirname)
    
    
def merge_mnist(dirname):
    outdir = dirname
    for idx, file in enumerate(os.listdir(dirname)):

        if '.DS_Store' in file:
            continue

        for nest in crawl_directory(dirname + file):

            index = 1

            if 'dims' in nest.split('/')[5]:
                outdir = '/'.join(nest.split('/')[:6]) + '/'
            else:
                # Uncomment the + for mixture
                outdir = dirname + nest.split('/')[7] + '/'
    

            # Initialize directory
            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            try:
                shutil.move(nest, outdir + 'trial_{0}/'.format(index))
            except:
                extension = nest.split('/')[-1]
                while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                    index += 1

                if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                    os.makedirs(outdir + 'trial_{0}/'.format(index))

                shutil.move(nest, outdir + 'trial_{0}/'.format(index))

    remove_empty_dirs(dirname)

In [4]:
dirname = '/Users/sob/Desktop/TO_ADD/'
outdir = dirname
for idx, file in enumerate(os.listdir(dirname)):

    if '.DS_Store' in file:
        continue

    for nest in crawl_directory(dirname + file):

        index = 1

        if 'dims' not in nest.split('/')[6]:
            outdir = '/'.join(nest.split('/')[:6]) + '/'# + nest.split('/')[8:9]) + '/'
        else:
            # Uncomment the + for mixture
            outdir = dirname + nest.split('/')[6] + '/'
            
        # Initialize directory
        if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
            os.makedirs(outdir + 'trial_{0}/'.format(index))

        try:
            shutil.move(nest, outdir + 'trial_{0}/'.format(index))
        except:
            extension = nest.split('/')[-1]
            while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                index += 1

            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            shutil.move(nest, outdir + 'trial_{0}/'.format(index))

remove_empty_dirs(dirname)

In [18]:
# dirname = '/Users/sob/Desktop/gan_results/hypertuning/multivariate/'
for file in os.listdir(dirname):
    if '.DS_Store' in file:
        continue
    
    print(file, len(os.listdir(dirname + file)))
    idx = 0
    
#     for f in os.listdir(dirname + file):
#         if '.DS_Store' in f:
#             continue
#         if len(os.listdir(dirname + file + '/' + f)) >= 60:
#             idx += 1
#             print(f, len(os.listdir(dirname + file + '/' + f)))
#         else:
#             print(f, len(os.listdir(dirname + file + '/' + f)))
# #             shutil.rmtree(dirname + file + '/' + f)
#     print('{0}/20'.format(idx))
#     print('\n')

trial_1 60
trial_10 60
trial_11 60
trial_12 60
trial_13 60
trial_14 60
trial_15 60
trial_16 60
trial_17 60
trial_18 60
trial_19 49
trial_2 60
trial_20 1
trial_21 1
trial_3 60
trial_4 60
trial_5 60
trial_6 60
trial_7 60
trial_8 60
trial_9 60


In [16]:
dirname = '/Users/sob/Desktop/updated_mnist/0_dims_0_samples/'
outdir = dirname
for idx, file in enumerate(os.listdir(dirname)):

    if '.DS_Store' in file:
        continue

    for nest in crawl_directory(dirname + file):

        index = 1

        if 'dims' in nest.split('/')[5]:
            outdir = '/'.join(nest.split('/')[:6]) + '/'
        else:
            # Uncomment the + for mixture
            outdir = dirname + nest.split('/')[7] + '/'

        # Initialize directory
        if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
            os.makedirs(outdir + 'trial_{0}/'.format(index))

        try:
            shutil.move(nest, outdir + 'trial_{0}/'.format(index))
        except:
            extension = nest.split('/')[-1]
            while os.path.exists(outdir + 'trial_{0}/'.format(index) + extension):
                index += 1

            if not os.path.exists(outdir + 'trial_{0}/'.format(index)):
                os.makedirs(outdir + 'trial_{0}/'.format(index))

            shutil.move(nest, outdir + 'trial_{0}/'.format(index))

remove_empty_dirs(dirname)

In [107]:
def get_best_performance_multivariate(mypath='/Users/sob/Desktop/mnist_new/0_dims_0_samples/trial_1/'):
    """ For a trial, get the best performance for multivariate data """
    # Get path, files in path
    files = os.listdir(mypath)
    results = []

    # Read in the files
    for file in files:
        if file == '.DS_Store':
            continue
                            
        with open(mypath + file, 'r') as f:
            data = json.load(f)

        results.append(data)
        
    # Initialize best dictionary
    optimal = nested_pickle_dict()

    # Go through all models, distributionss, metrics, and record the best
    for result in results:
        for model, distributions in result.items():
            for distribution, metrics in distributions.items():
                for metric, values in metrics.items():
                    if metric not in ["LR", "HDIM", "BSIZE"]:

                        # If metric is seen for the first time, it is the best
                        if metric not in optimal[model][distribution]:
                            optimal[model][distribution][metric]["value"] = values
                            optimal[model][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]

                        # Otherwise, compare it the presently considered value
                        elif min(optimal[model][distribution][metric]["value"]) > min(values):
                            optimal[model][distribution][metric]["value"] = values
                            optimal[model][distribution][metric]["parameters"] = [metrics["LR"], metrics["HDIM"], metrics["BSIZE"]]

    return optimal


In [109]:
# Read in the files
mypath = '/Users/sob/Desktop/mnist_new/trial_18/'#mnist_new/0_dims_0_samples/trial_1/'
files = os.listdir(mypath)
for file in files:
    if file == '.DS_Store':
        continue

    with open(mypath + file, 'r') as f:
        data = json.load(f)

    results.append(data)

In [115]:
optimal = get_best_performance_multivariate('/Users/sob/Desktop/mnist_new/0_dims_0_samples/trial_18/')

In [124]:
len(os.listdir('/Users/sob/Desktop/mnist_new/0_dims_0_samples/trial_17/'))

60

In [116]:
with open(best_path + '/trial_{0}.json'.format(18), 'w') as outfile:
    json.dump(optimal, outfile)

In [66]:
optimal = get_best_performance_multivariate()

In [64]:
from collections import defaultdict

def nested_pickle_dict():
    """ Picklable defaultdict nested dictionaries """
    return defaultdict(nested_pickle_dict)

In [112]:
import tqdm
best_path = '/Users/sob/Desktop/mnist_new/best/'
# if not os.path.exists(best_path):
#     os.makedirs(best_path)

# files = os.listdir('/Users/sob/Desktop/mnist_new/0_dims_0_samples/')
# for idx, f in tqdm.tqdm_notebook(enumerate(files)):
#     if f == '.DS_Store':
#         continue
    
#     optimal = get_best_performance_multivariate('/Users/sob/Desktop/mnist_new/0_dims_0_samples/' + f + '/')
    
#     with open(best_path + '/trial_{0}.json'.format(idx), 'w') as outfile:
#         json.dump(optimal, outfile)